In [ ]:
#Advent of Code Problem 4
#...I dont know why I wasnt using functions before...
#of course we now have two data type in one input....

import requests
import numpy as np


def getPuzzleInputAsList(day):
    cookie_file = open("../keys/session.cookie")
    cookie_raw = cookie_file.readline()
    cookie = cookie_raw.strip("\n")
    sesh = requests.Session()
    url = "https://adventofcode.com/2021/day/"+str(day)+"/input"
    req = sesh.get(url,cookies={"session":cookie})
    input_raw = req.text
    input_list = input_raw.split("\n")
    return input_list

#parse winning numbers out into a flat array
def getWinningNums(input_list):
    raw_winning = input_list[0]
    winning = raw_winning.split(",")
    return np.array(winning,dtype=np.int64)

#parse boards into a 3d shape
def getGameBoards(input_list):
    raw_gameboards = np.delete(input_list,np.argwhere(input_list == '')) #litty: https://thispointer.com/delete-elements-from-a-numpy-array-by-value-or-conditions-in-python/
    gameboards = np.array([],dtype=np.int64)
    for ea in raw_gameboards:
        gameboard_spaces = np.array(ea.split(" "))
        gameboard_num = np.array(np.delete(gameboard_spaces,np.argwhere(gameboard_spaces == '')),dtype=np.int64)
        gameboards = np.append(gameboards,gameboard_num)
    
    gameboards = np.reshape(gameboards,(-1,5,5))
    return gameboards

#data was coming in as string so this wasnt working, parsed data to int during GB creation; works on entire array now
def replaceWinner(gameboard,winner):
    gameboard = np.where(gameboard == winner, 0, gameboard)
    return gameboard

#https://stackoverflow.com/questions/69601220/bingo-using-python
#tests for lines of 0's in bingo card
def isWinner(gameboard):
    for i in range (5):
        row_zeros=np.count_nonzero(gameboard[i:,])#would like to understand this : notation
        col_zeros=np.count_nonzero(gameboard[:,i])
        if not row_zeros or not col_zeros: #check if we have 0 non-zeros, if line with non-zeros is discovered continue
            return (True)
    diagonal_zeros=np.count_nonzero(np.diag(gameboard))
    diagonal1_zeros=np.count_nonzero(np.diag(np.fliplr(gameboard)))
    if not diagonal_zeros or not diagonal1_zeros:
        return(True)
    return(False)


def bingoSubsystem(gameboards,numbers):
    winner = False
    winning_board = np.array([])
    for ea in winning_nums:
        win_num = ea
        if winner:
            break
        board_counter = 0
        gameboards = replaceWinner(gameboards,win_num)
        for ea in gameboards:
            winner = isWinner(ea)
            if winner:
                winning_board_index = board_counter
                winning_board = ea
                return win_num,winning_board_index,winning_board
            board_counter+=1
    return "No winning board??"

input_list = getPuzzleInputAsList(4) # this is done to reduce stress on AoC servers, only one call is needed

gameboards = getGameBoards(np.delete(input_list,0))#gets game boards after removing winning numbers from input_list

winning_nums = getWinningNums(input_list)

winning_details = bingoSubsystem(gameboards,winning_nums)
raw_winning_board = gameboards[winning_details[1]]
marked_winning_board = winning_details[2]
winning_number = winning_details[0]
print(winning_number)
print(raw_winning_board)
print(marked_winning_board)
print(winning_number*np.sum(marked_winning_board))

In [49]:
#Advent of Code Problem 4 part 2
#...I dont know why I wasnt using functions before...
#of course we now have two data type in one input....

#pretty sure I am running into an async issue, not all the winning game boards are being removed from the array
#this is leading to a false positive of a final gameboard when doing it the way I am
#we are also getting some game boards removed from the wrong indexes because of this..

#wont lie, looked up some solutions on reddit and damn.. 4d arrays that show every state... 
#I dont quite even know how they derive their solutions but their code is ~12 lines...
#I definately have a lot to learn about python, maybe all coding lol
#perhaps this is why I should have tried in c/java despite the compilation issue..
#jupyter is fun and its a great learning experience!!


import requests
import sys
import numpy as np
np.set_printoptions(threshold=sys.maxsize)


def getPuzzleInputAsList(day):
    cookie_file = open("../keys/session.cookie")
    cookie_raw = cookie_file.readline()
    cookie = cookie_raw.strip("\n")
    sesh = requests.Session()
    url = "https://adventofcode.com/2021/day/"+str(day)+"/input"
    req = sesh.get(url,cookies={"session":cookie})
    input_raw = req.text
    input_list = input_raw.split("\n")
    return input_list

#parse winning numbers out into a flat array
def getWinningNums(input_list):
    raw_winning = input_list[0]
    winning = raw_winning.split(",")
    return np.array(winning,dtype=np.int64)

#parse boards into a 3d shape
def getGameBoards(input_list):
    raw_gameboards = np.delete(input_list,np.argwhere(input_list == '')) #litty: https://thispointer.com/delete-elements-from-a-numpy-array-by-value-or-conditions-in-python/
    gameboards = np.array([],dtype=np.int64)
    for ea in raw_gameboards:
        gameboard_spaces = np.array(ea.split(" "))
        gameboard_num = np.array(np.delete(gameboard_spaces,np.argwhere(gameboard_spaces == '')),dtype=np.int64)
        gameboards = np.append(gameboards,gameboard_num)
    
    gameboards = np.reshape(gameboards,(-1,5,5))
    return gameboards

#data was coming in as string so this wasnt working, parsed data to int during GB creation; works on entire array now
def replaceWinner(gameboard,winner):
    gameboard = np.where(gameboard == winner, 0, gameboard)
    return gameboard

#https://stackoverflow.com/questions/69601220/bingo-using-python
#tests for lines of 0's in bingo card, well great.. zero is one of our numbers..
def isWinner(gameboard):
    for i in range (5):
        row_zeros=np.count_nonzero(gameboard[i:,])#would like to understand this : notation
        col_zeros=np.count_nonzero(gameboard[:,i])
        if not row_zeros or not col_zeros: #check if we have 0 non-zeros, if line with non-zeros is discovered continue
            return (True)
    diagonal_zeros=np.count_nonzero(np.diag(gameboard))
    diagonal1_zeros=np.count_nonzero(np.diag(np.fliplr(gameboard)))
    if not diagonal_zeros or not diagonal1_zeros:
        return(True)
    return(False)


async def removeGameboards(gameboards):
    winning_board_indices = np.array([],dtype=np.int64)
    board_counter = 0
    for ea in gameboards:
        winner = isWinner(ea)
        if winner: #recall bingosubsystem with removal of winner?? led to async issues when checking boards.. So tried this
            winning_board_indices = np.append(winning_board_indices,board_counter)
            print("winner at: "+str(board_counter)+", adding to indices: ")
            print(winning_board_indices)
            print(ea)
            print(gameboards[board_counter])
            #gameboards = np.delete(gameboards,board_counter,axis=0)
            #print(gameboards)
        board_counter+=1
    return np.delete(gameboards,winning_board_indices,axis=0)

async def bingoSubsystem(gameboards,numbers):
    winner = False
    winning_board = np.array([],dtype=np.int64)
    winning_board_indices = np.array([],dtype=np.int64)
    win_num = -1
    for win_num in winning_nums:
        print(np.shape(gameboards),win_num)
        if np.shape(gameboards)[0]<=1:
            print(gameboards)
            return win_num,winning_board_indices,winning_board
        gameboards = replaceWinner(gameboards,win_num)
        
        gameboards = await removeGameboards(gameboards)
        print(gameboards)
        
    print("finished our numbers")
    return win_num,winning_board_indices,winning_board

input_list = getPuzzleInputAsList(4) # this is done to reduce stress on AoC servers, only one call is needed

gameboards = getGameBoards(np.delete(input_list,0))#gets game boards after removing winning numbers from input_list

winning_nums = getWinningNums(input_list)

print(await bingoSubsystem(gameboards,winning_nums))
#print(winning_details)


(100, 5, 5) 94
[[[49 74 83 34 40]
  [87 16 57 75  3]
  [68  0 77 78 89]
  [56 38 29 26 60]
  [41 42 45 19  1]]

 [[42 35 10 20  9]
  [49 39 40 41 73]
  [ 3 48 91 81 88]
  [59 55 82 58 71]
  [61 51 17 26 72]]

 [[31 49 21 84 83]
  [18 86 53 75 29]
  [85  2 51 76 52]
  [48 28 24 69 12]
  [ 5 87 67 95 82]]

 [[54 21  0 63 13]
  [84 29 27 12 82]
  [55 86 33 90 95]
  [72 96 24 88 37]
  [38 51 35 46 50]]

 [[24  1 23 62 97]
  [53 72 99 59 81]
  [54 26 93 63 20]
  [79 41  2 86 98]
  [84 13 87 33 96]]

 [[36 85 51 32 84]
  [41 70 65 86 73]
  [ 0 28 80 81 59]
  [96 82  7 10 83]
  [ 8 21 29 91 16]]

 [[56 36 87 10 35]
  [ 8 90 37 22 96]
  [24 82 30 50 86]
  [70 52 55 51 57]
  [26 41 61 46 65]]

 [[ 6 28 71 21 50]
  [61 92 19 17 79]
  [85 69 63 32 41]
  [ 8 36 37 38 83]
  [13 45 88 77 78]]

 [[55 61 34 72 65]
  [ 8 92 39 27 86]
  [69 16 66  0 53]
  [35 41 50  1 42]
  [31 43  3 85 59]]

 [[55 78 47 85 50]
  [80 15 98 42 63]
  [ 2 68 37 24 45]
  [ 8 99 33 89 20]
  [35 28 60  5 34]]

 [[76  4 33 91 

[[[ 0 74 83  0  0]
  [87  0 57 75  3]
  [68  0 77 78 89]
  [ 0 38 29 26 60]
  [ 0 42 45 19  0]]

 [[42 35 10  0  9]
  [ 0  0  0  0  0]
  [ 3  0 91  0 88]
  [59  0 82  0 71]
  [61 51  0 26  0]]

 [[31  0  0  0 83]
  [18  0 53 75 29]
  [ 0  0 51 76 52]
  [ 0  0  0 69 12]
  [ 5 87 67 95 82]]

 [[54  0  0  0 13]
  [ 0 29 27 12 82]
  [ 0  0 33  0 95]
  [ 0 96  0 88 37]
  [38 51 35  0 50]]

 [[ 0  0  0  0  0]
  [53  0 99 59  0]
  [54 26 93  0  0]
  [79  0  0  0  0]
  [ 0 13 87 33 96]]

 [[ 0  0 51 32  0]
  [ 0 70 65  0  0]
  [ 0  0 80  0 59]
  [96 82  0 10 83]
  [ 8  0 29 91  0]]

 [[ 0  0 87 10 35]
  [ 8  0 37 22 96]
  [ 0 82 30 50  0]
  [70 52  0 51 57]
  [26  0 61  0 65]]

 [[ 0  0 71  0 50]
  [61  0 19  0 79]
  [ 0 69  0 32  0]
  [ 8  0 37 38 83]
  [13 45 88 77 78]]

 [[ 0 61  0  0 65]
  [ 8  0  0 27  0]
  [69  0  0  0 53]
  [35  0 50  0 42]
  [31 43  3  0 59]]

 [[ 0 78  0  0 50]
  [80 15  0 42  0]
  [ 0 68 37  0 45]
  [ 8 99 33 89  0]
  [35  0 60  5  0]]

 [[76  0 33 91  0]
  [ 0  0  0

In [ ]:
#used as a notepad again lol

def bingoSubsystem(gameboards,numbers):
    winner = False
    winning_board = np.array([],dtype=np.int64)
    winning_board_indices = np.array([],dtype=np.int64)
    win_num = -1
    for ea in winning_nums:
        win_num = ea
        print(np.shape(gameboards))
        print(winner)
        if np.shape(gameboards)[0] == 1:
            return win_num,winning_board_index,winning_board
        if winner:
            print("removing board at index:"+str(winning_board_indices))
            #print(gameboards)
            gameboards = np.delete(gameboards,0,axis=0)
            print(gameboards)
            winning_board_indices = np.array([],dtype=np.int64)
            winner = False
        board_counter = 0
        gameboards = replaceWinner(gameboards,win_num)
        for ea in gameboards:
            winner = isWinner(ea)
            if winner:
                print("wining ticket at: "+str(board_counter))
                winning_board_indices = np.append(winning_board_indices,board_counter)
                print(winning_board_indices)
            board_counter+=1
    print("finished our numbers")
    return win_num,winning_board_indices,winning_board



for ea in gameboards:
            print("testing for winner: "+str(board_counter))
            print(ea)
            winner = isWinner(ea)
            print(winner)
            if winner: #recall bingosubsystem with removal of winner??
                print(gameboards)
                print("winner at: "+str(board_counter)+", deleting board: ")
                print(ea)
                print(gameboards)
                #return bingoSubsystem(np.delete(gameboards,board_counter,axis=0),winning_nums)
            board_counter+=1